# Scraper

In [58]:
import pandas as pd 
import requests
import unicodedata
from tqdm import tqdm
from bs4 import BeautifulSoup

def get_links(): 
    response = requests.get('https://en.wikipedia.org/wiki/List_of_language_names')
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    # links = [link.get('href') for link in links if (link.get('title') is not None) and ('language' in link.get('title'))]
    links = [link.get('href') for link in links if (link.get('title') is not None)]
    # links = [link for link in links if link.endswith('_language')]
    links = ["https://en.wikipedia.org"+link for link in links]
    return links

['https://en.wikipedia.org/wiki/Hong_Kong_Special_Administration_Region',
 'https://en.wikipedia.org/wiki/States_and_federal_territories_of_Malaysia',
 'https://en.wikipedia.org/wiki/Adai_language',
 'https://en.wikipedia.org/wiki/Northern_Ireland',
 'https://en.wikipedia.org/wiki/Kapampangan_language',
 'https://en.wikipedia.org/wiki/Vlax_Romani_language',
 'https://en.wikipedia.org/wiki/Tripura',
 'https://en.wikipedia.org/wiki/Nahuatl_language',
 'https://en.wikipedia.org/wiki/Trinidad',
 'https://en.wikipedia.org/wiki/Hmong_language',
 'https://en.wikipedia.org/wiki/Palawan',
 'https://en.wikipedia.org/wiki/Democratic_Republic_of_Timor-Leste',
 'https://en.wikipedia.org/wiki/Pidgin',
 'https://en.wikipedia.org/wiki/Karelia',
 'https://en.wikipedia.org/wiki/Bahawalpur_South_Punjab',
 'https://en.wikipedia.org/wiki/Kachin_State',
 'https://en.wikipedia.org/wiki/South_Asia',
 'https://en.wikipedia.org/wiki/Fur_languages',
 'https://en.wikipedia.org/wiki/Parya_language',
 'https://en.w

In [72]:


def find_index_of_headers(url):
    response        = requests.get(url)
    content         = response.content
    parser          = BeautifulSoup(content, 'html.parser')
    table           = parser.find('table', {'class': 'infobox'})
    included_headers = {}
    
    for i in range(0, len(table.find_all('th', {'class': 'infobox-label'}))):
        included_headers[unicodedata.normalize('NFKD', table.find_all('th', {'class': 'infobox-label'}).__getitem__(i).getText())] = i
    return included_headers


urls = get_links()
print(len(urls))
print(urls[:10])

urls = list(set(urls))

urls = [url for url in urls if url not in ['Official_language', 'Minority_language']]
urls = [url for url in urls if url.startswith('List_of_countries') == False]
urls = [url for url in urls if url.count(':') == 1]
urls = [url for url in urls if '?' not in url]

3540
['https://en.wikipedia.org/wiki/Main_Page', 'https://en.wikipedia.org/wiki/Wikipedia:Contents', 'https://en.wikipedia.org/wiki/Portal:Current_events', 'https://en.wikipedia.org/wiki/Special:Random', 'https://en.wikipedia.org/wiki/Wikipedia:About', 'https://en.wikipedia.org//en.wikipedia.org/wiki/Wikipedia:Contact_us', 'https://en.wikipedia.orghttps://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en', 'https://en.wikipedia.org/wiki/Help:Contents', 'https://en.wikipedia.org/wiki/Help:Introduction', 'https://en.wikipedia.org/wiki/Wikipedia:Community_portal']


In [10]:
urls = ['https://en.wikipedia.org/wiki/Aari_language']

In [73]:
data = []
errors = []

for url in tqdm(urls): 
    # try: 
        lang = url.split('/')[-1]

        # print(requests.get(url).status_code)
        # print(url)
        response        = requests.get(url)
        content         = response.content
        parser          = BeautifulSoup(content, 'html.parser')
        table           = parser.find('table', {'class': 'infobox'})

        if table == None:
             print("No table found ", url)
             continue

        header_indices  = find_index_of_headers(url)

        # print(header_indices)

        infobox_data = table.find_all('td', {'class': 'infobox-data'})

        regions         = infobox_data[header_indices['Region']].text              if 'Region' in header_indices else None
        off_lang        = infobox_data[header_indices['Official language in']].text if 'Official language in' in header_indices else None
        rec_min_lang    = infobox_data[header_indices['Recognised minoritylanguage in']].text if 'Recognised minoritylanguage in' in header_indices else None
        speakers        = infobox_data[header_indices['Speakers']].text             if 'Speakers' in header_indices else None
        iso3code        = infobox_data[header_indices['ISO 639-3']].text            if 'ISO 639-3' in header_indices else None

        family          = [x for x in infobox_data[header_indices['Language family']].get_text('title').split('title') if x != "\n"]\
                            if 'Language family' in header_indices else None
        
        dialects        = [x for x in infobox_data[header_indices['Dialects']].get_text('title').split('title') if x != "\n"] \
                            if 'Dialects' in header_indices else None
        
        data.append((lang, family, dialects, iso3code, speakers, regions, off_lang, rec_min_lang))
    # except Exception as e: 
    #     errors.append((e, url))
    #     continue

df = pd.DataFrame(data, columns=['lang', 'family', 'dialects', 'iso3code', 'speakers', 'regions', 'off_lang', 'rec_min_lang'])


  0%|          | 2/1389 [00:01<15:03,  1.54it/s]

No table found  https://en.wikipedia.org/wiki/States_and_federal_territories_of_Malaysia


  1%|          | 13/1389 [00:11<19:08,  1.20it/s]

No table found  https://en.wikipedia.org/wiki/Pidgin


  1%|          | 14/1389 [00:11<15:44,  1.46it/s]

...

 98%|█████████▊| 1359/1389 [20:35<00:28,  1.05it/s]

No table found  https://en.wikipedia.org/wiki/Al-Andalus


 99%|█████████▉| 1376/1389 [20:54<00:09,  1.35it/s]

No table found  https://en.wikipedia.org/wiki/List_of_countries_where_Spanish_is_an_official_language


100%|██████████| 1389/1389 [21:06<00:00,  1.10it/s]


In [79]:
(df.isna().sum()  / len(df)).sort_values()

lang            0.000000
iso3code        0.564662
family          0.572932
regions         0.630827
off_lang        0.812030
dialects        0.839850
rec_min_lang    0.883459
speakers        0.976692
dtype: float64

In [91]:
languages = df[~df['iso3code'].isna()]

In [94]:
languages.isna().sum().sort_values()/len(languages)

lang            0.000000
iso3code        0.000000
family          0.018998
regions         0.265976
off_lang        0.568221
dialects        0.632124
rec_min_lang    0.732297
speakers        0.946459
dtype: float64

In [98]:
languages[:20]

,lang,family,dialects,iso3code,speakers,regions,off_lang,rec_min_lang
1,Adai_language,[unclassified],None,xad,None,Louisiana,None,None
3,Kapampangan_language,"[Austronesian, Malayo-Polynesian, Philippine, ...",None,pam,None,"Central Luzon (entirety of Pampanga, southern ...",Angeles City[3][4][5],Regional language of the Philippines
4,Vlax_Romani_language,"[Indo-European, Indo-Iranian, Indo-Aryan, West...",None,rmy,None,None,None,"Hungary, Romania, Poland, and Serbia"
6,Nahuatl_language,"[Uto-Aztecan, Southern Uto-Aztecan, Nahuan, Na...","[Western Peripheral Nahuatl, Eastern Periphera...","nhe Huasteca NahuatlFor other varieties, see N...",None,Mexico: Puebla Veracruz Hidalgo Guerrero San L...,Mexico (through the General Law of Linguistic...,None
8,Hmong_language,"[Hmong–Mien, Hmongic, Core Hmongic, West Hmong...",None,hmn – inclusive code for the Hmong/Mong macrol...,None,None,None,China Laos Myanmar Vietnam Thailand
15,Parya_language,"[Indo-European, Indo-Iranian, Indo-Aryan, Cent...",None,paq,None,"Gissar Valley, Surkhandarya basin",None,None
17,Arin_language,"[Dené–Yeniseian, ?\n, Yeniseian, Southern Yeni...",None,xrn,None,Yenisei River,None,None
19,Latvian_language,"[Indo-European, Balto-Slavic, Baltic, East Bal...",None,lav – inclusive codeIndividual codes:lvs – Sta...,None,Baltic,Latvia European Union,None
21,Odia_language,"[Indo-European, Indo-Iranian, Indo-Aryan, East...","[Northern, , Central, , Southern, , , Northwes...",ori – inclusive codeIndividual codes:ory – Odi...,None,Odisha[a],India\nOdisha\nJharkhand[3] (additional)\nWes...,None
22,Caddo_language,"[Caddoan, Caddo]",None,cad,None,Caddo County in western Oklahoma,None,None
